# **ComfyUI Qwen Image-Edit 2509 + UltimateSDUpscale Cloud Pipeline**

# **=== Installing Dependencies ===**

In [ ]:
!apt -y update -qq
!apt -y install -qq libgl1-mesa-glx wget git
!npm install -g localtunnel
!pip install --upgrade gguf

In [ ]:
import os,torch,subprocess,time, sys, threading 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# ============================================================
# Installing COMFYUI TO /kaggle/temp
# ============================================================
BASE_DIR = "/kaggle/temp/ComfyUI"
if not os.path.exists(BASE_DIR):
    !git clone https://github.com/comfyanonymous/ComfyUI.git {BASE_DIR}
else:
    print("ComfyUI already cloned at", BASE_DIR)

%cd {BASE_DIR}
!pip install -r requirements.txt --quiet

In [ ]:
# ============================================================
# Installing COMFYUI Nodes and Extra Features (No Auth Prompt)
# ============================================================
CUSTOM_NODES_DIR = os.path.join(BASE_DIR, "custom_nodes")
os.makedirs(CUSTOM_NODES_DIR, exist_ok=True)

NODE_REPOS = {
    "Comfyui-QwenEditUtils": "https://github.com/lrzjason/Comfyui-QwenEditUtils.git",
    "ComfyUI_UltimateSDUpscale": "https://github.com/ssitu/ComfyUI_UltimateSDUpscale.git",
    "rgthree-comfy": "https://github.com/rgthree/rgthree-comfy.git",
    "ComfyUI-GGUF": "https://github.com/city96/ComfyUI-GGUF",
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "comfyui-inspire-pack": "https://github.com/ltdrdata/comfyui-inspire-pack.git"
}

for folder, repo_url in NODE_REPOS.items():
    target_dir = os.path.join(CUSTOM_NODES_DIR, folder)
    if not os.path.exists(target_dir):
        print(f" Cloning {folder} ...")
        !git clone --depth=1 --quiet "{repo_url}" "{target_dir}" || (sleep 5 && git clone --depth=1 --quiet "{repo_url}" "{target_dir}")
        print(f" {folder} installed")
    else:
        print(f" {folder} already exists")

In [ ]:
# ============================================================
# CREATE REQUIRED FOLDERS 
# ============================================================
REQUIRED_FOLDERS = [
    'models/checkpoints',
    'models/controlnet',
    'models/vae',
    'models/upscale_models',
    'models/clip',
    'models/unet',
    'custom_nodes',
    'input',
    'output',
    'temp',
    'user'
]

for folder in REQUIRED_FOLDERS:
    os.makedirs(os.path.join(BASE_DIR, folder), exist_ok=True)

print(f" Folder structure ready at: {BASE_DIR}")

In [ ]:
# ============================================================
# GPU CHECK
# ============================================================
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

# === Installing Qwen-Image-Edit-2509 ===

In [ ]:
# ============================================================
# DOWNLOAD Main MODEL FROM HUGGING FACE (to models/unet)
# ============================================================
model_url = "https://huggingface.co/QuantStack/Qwen-Image-Edit-2509-GGUF/resolve/main/Qwen-Image-Edit-2509-Q3_K_M.gguf?download=true"
model_filename = "Qwen-Image-Edit-2509-Q3_K_M.gguf"
model_dir = os.path.join(BASE_DIR, "models/unet")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" Model already exists: {model_filename}")

In [ ]:
# ============================================================
# DOWNLOAD Main Text Encoder FROM HUGGING FACE (to models/text_encoders)
# ============================================================
model_url = "https://huggingface.co/unsloth/Qwen2.5-VL-7B-Instruct-GGUF/resolve/main/Qwen2.5-VL-7B-Instruct-Q3_K_M.gguf?download=true"
model_filename = "Qwen2.5-VL-7B-Instruct-Q3_K_M.gguf"
model_dir = os.path.join(BASE_DIR, "models/text_encoders")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" Model already exists: {model_filename}")

In [ ]:
# ============================================================
# DOWNLOAD Text_Encoder (mmproj) FROM HUGGING FACE (to models/text_encoders)
# ============================================================
model_url = "https://huggingface.co/unsloth/Qwen2.5-VL-7B-Instruct-GGUF/resolve/main/mmproj-BF16.gguf?download=true"
model_filename = "Qwen2.5-VL-7B-Instruct-mmproj-BF16.gguf"
model_dir = os.path.join(BASE_DIR, "models/text_encoders")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" Model already exists: {model_filename}")

In [ ]:
# ============================================================
# DOWNLOAD VAE FROM HUGGING FACE (to models/vae)
# ============================================================
model_url = "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors?download=true"
model_filename = "qwen_image_vae.safetensors"
model_dir = os.path.join(BASE_DIR, "models/vae")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" VAE already exists: {model_filename}")

In [ ]:
# ============================================================
# DOWNLOAD Qwen-Image-Lightning FROM HUGGING FACE (to models/loras)
# ============================================================
model_url = "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-8steps-V2.0.safetensors?download=true"
model_filename = "Qwen-Image-Lightning-8steps-V2.0.safetensors"
model_dir = os.path.join(BASE_DIR, "models/loras")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" Model already exists: {model_filename}")

In [ ]:
# ============================================================
# DOWNLOAD Transfer Lora (to models/loras)
# ============================================================
model_url = "https://civitai.com/api/download/models/2196278?type=Model&format=SafeTensor"
model_filename = "Transfer_Qwen_Image_Edit_2509.safetensors"
model_dir = os.path.join(BASE_DIR, "models/loras")
model_path = os.path.join(model_dir, model_filename)

if not os.path.exists(model_path):
    print(f" Downloading {model_filename} ...")
    !wget -q -O "{model_path}" "{model_url}"
    print(f" Saved to {model_path}")
else:
    print(f" Model already exists: {model_filename}")

# === Installing UltimateSDUpscale ===

In [ ]:
# ============================================================
# Install UltimateSDUpscale custom node
# ============================================================
ULTIMATE_PATH = os.path.join(BASE_DIR, "custom_nodes", "ComfyUI_UltimateSDUpscale")
!rm -rf "{ULTIMATE_PATH}"
!git clone --recursive https://github.com/ssitu/ComfyUI_UltimateSDUpscale "{ULTIMATE_PATH}" --quiet
print("Installed ComfyUI_UltimateSDUpscale")

In [ ]:
# ============================================================
# INSTALL Checkpoint for UltimateSDUpscale
# ============================================================
CHECKPOINT_DIR = os.path.join(BASE_DIR, "models", "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

checkpoint_url = "https://civitai.com/api/download/models/176425?type=Model&format=SafeTensor&size=pruned&fp=fp16"
checkpoint_name = "majicMIX.safetensors"
checkpoint_path = os.path.join(CHECKPOINT_DIR, checkpoint_name)

if not os.path.exists(checkpoint_path):
    print(f" Downloading checkpoint: {checkpoint_name}")
    !wget -q -O "{checkpoint_path}" "{checkpoint_url}"
    print(f" Checkpoint saved to {checkpoint_path}")
else:
    print(f" Checkpoint already exists: {checkpoint_path}")

In [ ]:
# ============================================================
# INSTALL Upscaler for UltimateSDUpscale
# ============================================================
UPSCALER_DIR = os.path.join(BASE_DIR, "models", "upscale_models")
os.makedirs(UPSCALER_DIR, exist_ok=True)
upscaler_url = "https://huggingface.co/gemasai/4x_NMKD-Superscale-SP_178000_G/resolve/main/4x_NMKD-Superscale-SP_178000_G.pth"
upscaler_name = "4x_NMKD-Superscale-SP_178000_G.pth"
upscaler_path = os.path.join(UPSCALER_DIR, upscaler_name)

if not os.path.exists(upscaler_path):
    print(f" Downloading upscaler model: {upscaler_name}")
    !wget -q -O "{upscaler_path}" "{upscaler_url}"
    print(f" Upscaler model saved to {upscaler_path}")
else:
    print(f" Upscaler model already exists: {upscaler_path}")

# === Launching ComfyUI ===

In [ ]:
# ============================================================
# START COMFYUI + STREAM LOGS TO CELL
# ============================================================

# Kill any stale processes BEFORE starting
os.system("fuser -k 8188/tcp || true")
os.system("pkill -f 'python.*main.py' || true")

def stream_process_output(proc):
    for line in iter(proc.stdout.readline, b''):
        if not line:
            break
        sys.stdout.write(line.decode(errors='ignore'))
        sys.stdout.flush()

def start_comfyui_singlegpu():
    # use a single GPU id (0)
    proc = subprocess.Popen(
        ["python", "/kaggle/temp/ComfyUI/main.py", "--listen", "0.0.0.0", "--port", "8188", "--cuda-device", "0"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT
    )
    stream_process_output(proc)

# start comfyui and stream logs in a background thread
thread = threading.Thread(target=start_comfyui_singlegpu, daemon=True)
thread.start()

# give it time to boot and stream initial logs
time.sleep(10)

# Start LocalTunnel (do NOT kill comfyui now)
print("\n Creating LocalTunnel link (wait a few seconds)...")
tunnel_proc = subprocess.Popen(["lt", "--port", "8188", "--allow-invalid-cert"],
                               stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

public_url = None
for _ in range(30):
    line = tunnel_proc.stdout.readline().decode(errors="ignore")
    if line:
        print(line.strip())
    if "your url is:" in line.lower():
        public_url = line.strip().split()[-1]
        print(" Public URL:", public_url)
        break
    time.sleep(1)

if not public_url:
    print(" Failed to detect LocalTunnel URL. Try re-running the cell.")
else:
    print("\n Fetching tunnel access password ...")
    os.system("curl -s https://loca.lt/mytunnelpassword || true")
    print("\n Done. Use the public URL above and password if prompted.")